# 中等项目2：医疗诊断辅助系统

## 项目描述

使用逻辑回归构建医疗诊断辅助系统，帮助医生进行疾病诊断。本项目将展示如何在实际医疗场景中应用逻辑回归。

## 学习目标

通过本项目，你将学会：
1. 如何处理医疗数据（特征选择、标准化）
2. 如何训练高准确率的诊断模型
3. 如何评估医疗模型的性能（敏感性、特异性、ROC曲线）
4. 如何分析特征重要性（哪些症状/指标最重要）
5. 如何解释模型结果（诊断规则、风险评估）

## 项目流程

1. 数据准备：加载医疗数据集，数据质量检查
2. 数据探索：统计分析、可视化、相关性分析
3. 特征工程：特征选择、标准化、缺失值处理
4. 模型训练：逻辑回归训练，超参数调优，交叉验证
5. 模型评估：敏感性、特异性、ROC曲线、混淆矩阵
6. 结果解释：特征重要性分析、诊断规则、风险评估


## 1. 环境准备


In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix,
                            roc_auc_score, roc_curve, precision_recall_curve, auc)
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
np.random.seed(42)
%matplotlib inline

print("环境准备完成！")


## 2. 数据加载与探索

加载乳腺癌数据集（作为医疗诊断的示例）。在实际应用中，你需要使用真实的医疗数据集。


In [ ]:
# 加载乳腺癌数据集
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

# 转换为DataFrame便于分析
df = pd.DataFrame(X, columns=cancer.feature_names)
df['diagnosis'] = y  # 0=恶性(Malignant), 1=良性(Benign)

print(f"数据集信息:")
print(f"  样本数量: {df.shape[0]}")
print(f"  特征数量: {df.shape[1] - 1}")
print(f"  类别: {cancer.target_names}")
print(f"\n类别分布:")
print(f"  恶性 (Malignant): {(y == 0).sum()} ({((y == 0).sum() / len(y)):.1%})")
print(f"  良性 (Benign): {(y == 1).sum()} ({((y == 1).sum() / len(y)):.1%})")
print(f"\n数据预览:")
print(df.head())
print(f"\n数据统计:")
print(df.describe())


## 3. 数据可视化

可视化数据分布和特征相关性。


In [ ]:
# 可视化特征分布
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 选择几个重要特征进行可视化
important_features = ['mean radius', 'mean texture', 'mean perimeter', 'mean area']

for idx, feature in enumerate(important_features):
    ax = axes[idx // 2, idx % 2]
    
    # 按类别分组绘制直方图
    df[df['diagnosis'] == 0][feature].hist(alpha=0.5, label='恶性', bins=20, ax=ax)
    df[df['diagnosis'] == 1][feature].hist(alpha=0.5, label='良性', bins=20, ax=ax)
    ax.set_xlabel(feature)
    ax.set_ylabel('频数')
    ax.set_title(f'{feature} 分布')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 特征相关性热力图（选择前10个特征）
plt.figure(figsize=(12, 10))
correlation_matrix = df[important_features + ['diagnosis']].corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('特征相关性热力图')
plt.tight_layout()
plt.show()


## 4. 数据预处理

准备训练数据：特征标准化、数据划分。


In [ ]:
# 准备特征和目标
X = df.drop('diagnosis', axis=1).values
y = df['diagnosis'].values

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 标准化特征（逻辑回归通常需要标准化）
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"训练集大小: {X_train.shape[0]}")
print(f"测试集大小: {X_test.shape[0]}")
print(f"特征数量: {X_train.shape[1]}")
print(f"\n训练集类别分布:")
print(f"  恶性: {(y_train == 0).sum()} ({((y_train == 0).sum() / len(y_train)):.1%})")
print(f"  良性: {(y_train == 1).sum()} ({((y_train == 1).sum() / len(y_train)):.1%})")


## 5. 模型训练

训练逻辑回归模型，使用交叉验证和网格搜索进行超参数调优。


In [ ]:
# 使用网格搜索进行超参数调优
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']  # liblinear支持L1和L2
}

# 创建基础模型
base_model = LogisticRegression(random_state=42, max_iter=1000)

# 网格搜索
print("进行网格搜索...")
grid_search = GridSearchCV(
    base_model, param_grid, cv=5, scoring='roc_auc', 
    n_jobs=-1, verbose=1
)
grid_search.fit(X_train_scaled, y_train)

print(f"\n最佳参数: {grid_search.best_params_}")
print(f"最佳交叉验证得分: {grid_search.best_score_:.4f}")

# 使用最佳参数训练模型
model = grid_search.best_estimator_
model.fit(X_train_scaled, y_train)

print("\n模型训练完成！")


## 6. 模型评估

评估模型性能，包括准确率、敏感性、特异性、ROC曲线等医疗诊断关键指标。


In [ ]:
# 预测
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)[:, 1]

# 计算评估指标
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

# 计算混淆矩阵
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

# 医疗诊断关键指标
sensitivity = tp / (tp + fn)  # 敏感性（召回率）：正确识别阳性的比例
specificity = tn / (tn + fp)  # 特异性：正确识别阴性的比例
precision = tp / (tp + fp)    # 精确率
f1 = 2 * (precision * sensitivity) / (precision + sensitivity)  # F1分数

print("=" * 60)
print("模型评估结果")
print("=" * 60)
print(f"准确率 (Accuracy): {accuracy:.4f}")
print(f"敏感性 (Sensitivity/Recall): {sensitivity:.4f}")
print(f"特异性 (Specificity): {specificity:.4f}")
print(f"精确率 (Precision): {precision:.4f}")
print(f"F1分数: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print("\n分类报告:")
print(classification_report(y_test, y_pred, target_names=['恶性', '良性']))
print("\n混淆矩阵:")
print(cm)


In [ ]:
# 可视化评估结果
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. 混淆矩阵
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['恶性', '良性'],
            yticklabels=['恶性', '良性'])
axes[0].set_title('混淆矩阵')
axes[0].set_ylabel('真实标签')
axes[0].set_xlabel('预测标签')

# 2. ROC曲线
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
axes[1].plot(fpr, tpr, label=f'ROC曲线 (AUC = {roc_auc:.2f})', linewidth=2)
axes[1].plot([0, 1], [0, 1], 'k--', label='随机分类器')
axes[1].set_xlabel('假正例率 (1-特异性)')
axes[1].set_ylabel('真正例率 (敏感性)')
axes[1].set_title('ROC曲线')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# 3. Precision-Recall曲线
precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_proba)
pr_auc = auc(recall_curve, precision_curve)
axes[2].plot(recall_curve, precision_curve, label=f'PR曲线 (AUC = {pr_auc:.2f})', linewidth=2)
axes[2].set_xlabel('召回率 (敏感性)')
axes[2].set_ylabel('精确率')
axes[2].set_title('Precision-Recall曲线')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 7. 特征重要性分析

分析哪些特征对诊断最重要，这对于医疗诊断的可解释性非常重要。


In [ ]:
# 获取特征重要性（系数的绝对值）
feature_importance = pd.DataFrame({
    'feature': cancer.feature_names,
    'coefficient': model.coef_[0],
    'abs_coefficient': np.abs(model.coef_[0])
}).sort_values('abs_coefficient', ascending=False)

print("特征重要性（按系数绝对值排序）:")
print(feature_importance.head(10))

# 可视化特征重要性
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(15)
plt.barh(range(len(top_features)), top_features['abs_coefficient'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('系数绝对值（重要性）')
plt.title('Top 15 最重要特征')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# 显示正负系数（正系数表示增加良性概率，负系数表示增加恶性概率）
print("\n特征对诊断的影响:")
print("正系数：增加良性概率（降低恶性风险）")
print("负系数：增加恶性概率（提高恶性风险）")
print("\nTop 10 特征:")
for idx, row in feature_importance.head(10).iterrows():
    direction = "良性" if row['coefficient'] > 0 else "恶性"
    print(f"  {row['feature']:<30}: {row['coefficient']:>8.4f} (倾向于 {direction})")


## 8. 模型应用示例

展示如何使用训练好的模型进行新患者的诊断预测。


In [ ]:
# 模拟新患者数据（使用测试集中的一个样本）
sample_idx = 0
new_patient = X_test_scaled[sample_idx:sample_idx+1]

# 预测
prediction = model.predict(new_patient)[0]
probability = model.predict_proba(new_patient)[0]

print("=" * 60)
print("新患者诊断预测")
print("=" * 60)
print(f"真实诊断: {'良性' if y_test[sample_idx] == 1 else '恶性'}")
print(f"预测诊断: {'良性' if prediction == 1 else '恶性'}")
print(f"\n诊断概率:")
print(f"  恶性概率: {probability[0]:.2%}")
print(f"  良性概率: {probability[1]:.2%}")

# 风险评估
if probability[1] > 0.7:
    risk_level = "低风险"
elif probability[1] > 0.4:
    risk_level = "中等风险"
else:
    risk_level = "高风险"

print(f"\n风险评估: {risk_level}")
print(f"\n注意: 本模型仅作为辅助诊断工具，不能替代专业医生的诊断！")


## 9. 总结与思考

### 项目总结

通过本项目，我们：
1. ✅ 成功构建了医疗诊断辅助系统
2. ✅ 实现了高准确率的诊断模型（准确率 > 0.90）
3. ✅ 分析了特征重要性，识别了关键诊断指标
4. ✅ 评估了模型的敏感性、特异性等医疗关键指标

### 关键知识点

1. **医疗诊断指标**：
   - 敏感性（Sensitivity）：正确识别阳性的比例，对医疗诊断非常重要
   - 特异性（Specificity）：正确识别阴性的比例
   - ROC曲线：评估模型在不同阈值下的性能

2. **特征重要性**：
   - 逻辑回归的系数可以解释特征对诊断的影响
   - 正系数表示增加良性概率，负系数表示增加恶性概率

3. **模型可解释性**：
   - 逻辑回归提供了可解释的诊断规则
   - 可以分析哪些症状/指标最重要

### 注意事项

⚠️ **重要提醒**：
- 本模型仅作为辅助诊断工具，不能替代专业医生的诊断
- 医疗数据需要谨慎处理，确保数据质量和隐私保护
- 模型需要经过专业医学知识验证
- 实际应用中需要考虑更多因素（患者历史、家族史等）

### 改进方向

1. 使用更多医疗特征（患者历史、家族史、生活习惯等）
2. 处理类别不平衡问题（如果存在）
3. 集成多个模型提高准确率
4. 开发用户友好的诊断界面
5. 持续更新模型以适应新的医疗数据
